In [ ]:
!ls

'Online Retail.xlsx'	'Online Retail.xlsx.2'
'Online Retail.xlsx.1'	 sample_data


# Практическое задание №6

## Задание 1
Используйте датасет с  практики текущего урока.

In [ ]:
!wget https://github.com/krashivan/working_files/raw/main/PA2HW4/Online%20Retail.xlsx

--2021-12-27 20:01:52--  https://github.com/krashivan/working_files/raw/main/PA2HW4/Online%20Retail.xlsx
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/krashivan/working_files/main/PA2HW4/Online%20Retail.xlsx [following]
--2021-12-27 20:01:52--  https://raw.githubusercontent.com/krashivan/working_files/main/PA2HW4/Online%20Retail.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23715344 (23M) [application/octet-stream]
Saving to: ‘Online Retail.xlsx.3’

Online Retail.xlsx. 100%[===================>]  22.62M   137MB/s    in 0.2s    

2021-12-27 20:01:53 (137 MB/s) - ‘Online Retail.xlsx.3’ sa

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_excel('Online Retail.xlsx')

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## Задание №2
Создайте матрицу item-customer (по срокам - товары, по столбцам - покупатели)

In [ ]:
df = df.loc[df['Quantity'] > 0].dropna(subset=['StockCode', 'CustomerID'])

df_item_customer = df.pivot_table(values = 'Quantity', index = 'StockCode', columns = 'CustomerID', aggfunc = 'sum')
df_item_customer = df_item_customer.applymap(lambda x: 1 if x > 0 else 0)
df_item_customer.head()

CustomerID,12346.0,12347.0,12348.0,12349.0,12350.0,12352.0,12353.0,12354.0,12355.0,12356.0,12357.0,12358.0,12359.0,12360.0,12361.0,12362.0,12363.0,12364.0,12365.0,12367.0,12370.0,12371.0,12372.0,12373.0,12374.0,12375.0,12377.0,12378.0,12379.0,12380.0,12381.0,12383.0,12384.0,12386.0,12388.0,12390.0,12391.0,12393.0,12394.0,12395.0,...,18230.0,18231.0,18232.0,18233.0,18235.0,18236.0,18237.0,18239.0,18240.0,18241.0,18242.0,18245.0,18246.0,18248.0,18249.0,18250.0,18251.0,18252.0,18255.0,18257.0,18259.0,18260.0,18261.0,18262.0,18263.0,18265.0,18268.0,18269.0,18270.0,18272.0,18273.0,18274.0,18276.0,18277.0,18278.0,18280.0,18281.0,18282.0,18283.0,18287.0
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10080,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10120,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10133,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Задание №3
Проведите оценку мер близости товаров, получив матрицу item_item_sim_matrix со значениями косинусов между векторами товаров.

In [ ]:
m = df_item_customer.values

from scipy.sparse import csr_matrix

def cos_sim(array):
  csr_array = csr_matrix(array)
  numerator = np.dot(csr_array,csr_array.T).toarray()
  denomenator = np.linalg.norm(array, axis = 1, keepdims = True)*np.linalg.norm(array.T, axis = 0, keepdims = True)
  return numerator / denomenator

item_item_sim_matrix = cos_sim(m)

item_item_sim_matrix = pd.DataFrame(item_item_sim_matrix)
item_item_sim_matrix.columns = df_item_customer.index

item_item_sim_matrix['StockCode'] = df_item_customer.index
item_item_sim_matrix = item_item_sim_matrix.set_index('StockCode')

item_item_sim_matrix.head()

StockCode,10002,10080,10120,10125,10133,10135,11001,15030,15034,15036,15039,16008,16010,16011,16012,16014,16015,16016,16033,16043,16045,16046,16048,16049,16052,16054,16216,16218,16219,16225,16235,16236,16237,16238,16254,16259,17001,17003,17021,17038,...,90209B,90209C,90210A,90210B,90210C,90210D,90211A,90211B,90212B,90212C,90214A,90214B,90214C,90214D,90214E,90214F,90214G,90214H,90214I,90214J,90214K,90214L,90214M,90214N,90214O,90214P,90214R,90214S,90214T,90214U,90214V,90214W,90214Y,90214Z,BANK CHARGES,C2,DOT,M,PADS,POST
StockCode,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10002,1.000000,0.000000,0.094868,0.090351,0.062932,0.098907,0.095346,0.047673,0.075593,0.090815,0.062284,0.043033,0.182574,0.076948,0.027524,0.054233,0.036274,0.049386,0.0,0.0,0.040489,0.043853,0.024693,0.000000,0.070711,0.058722,0.076948,0.072336,0.049386,0.062828,0.095871,0.102923,0.106012,0.080978,0.000000,0.119523,0.0,0.094907,0.115470,0.000000,...,0.0,0.055902,0.000000,0.000000,0.000000,0.000000,0.091287,0.0,0.0,0.0,0.000000,0.0,0.000000,0.070711,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.029361,0.0,0.066915,0.000000,0.078217
10080,0.000000,1.000000,0.000000,0.032774,0.045655,0.047836,0.000000,0.000000,0.082261,0.049413,0.030124,0.062439,0.000000,0.037216,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.117495,0.000000,0.035829,0.102598,0.000000,0.000000,0.037216,0.069971,0.107486,0.060774,0.027821,0.029867,0.065922,0.058747,0.000000,0.043355,0.0,0.019672,0.041885,0.108148,...,0.0,0.081111,0.000000,0.000000,0.102598,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.162221,0.102598,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.016182,0.000000,0.000000
10120,0.094868,0.000000,1.000000,0.057143,0.059702,0.041703,0.060302,0.060302,0.095618,0.028718,0.026261,0.027217,0.115470,0.032444,0.139262,0.068599,0.045883,0.031235,0.0,0.0,0.076822,0.110940,0.031235,0.000000,0.000000,0.111417,0.162221,0.060999,0.062470,0.185435,0.121268,0.234340,0.344817,0.230466,0.000000,0.113389,0.0,0.051450,0.036515,0.047140,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.115470,0.0,0.0,0.0,0.000000,0.0,0.000000,0.089443,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.010993
10125,0.090351,0.032774,0.057143,1.000000,0.042644,0.044682,0.043073,0.000000,0.051224,0.030770,0.056274,0.077762,0.000000,0.069523,0.049736,0.024500,0.000000,0.000000,0.0,0.0,0.091455,0.039621,0.000000,0.063888,0.000000,0.053056,0.046349,0.000000,0.000000,0.018922,0.121268,0.130189,0.136832,0.109746,0.053995,0.053995,0.0,0.036750,0.000000,0.000000,...,0.0,0.000000,0.000000,0.058321,0.000000,0.071429,0.000000,0.0,0.0,0.0,0.063888,0.0,0.053995,0.063888,0.058321,0.000000,0.000000,0.0,0.0,0.0,0.058321,0.071429,0.0,0.0,0.0,0.071429,0.0,0.063888,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.070535,0.000000,0.070669
10133,0.062932,0.045655,0.059702,0.042644,1.000000,0.280097,0.045002,0.060003,0.071358,0.057152,0.052262,0.121867,0.057448,0.080708,0.051964,0.068259,0.068483,0.093239,0.0,0.0,0.127401,0.082792,0.093239,0.000000,0.133498,0.203251,0.064567,0.121393,0.139859,0.065898,0.084466,0.116589,0.104838,0.101921,0.037609,0.094022,0.0,0.068259,0.054500,0.046907,...,0.0,0.035180,0.057448,0.040622,0.000000,0.000000,0.057448,0.0,0.0,0.0,0.000000,0.0,0.000000,0.044499,0.000000,0.070360,0.044499,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.036955,0.0,0.070185,0.049752,0.021877


## Задание №4
Отберите ТОП-10 похожих товаров по StockCode 

Исходный товар - StockCode: 23166 Description: MEDIUM CERAMIC TOP STORAGE JAR

In [ ]:
top10_index_list = item_item_sim_matrix.loc[23166].sort_values(ascending=False).head(12)[1:12].index.tolist()

top10_index_list

[23165, 23167, 22993, 23307, 22722, 22720, 22666, 23243, 22961, 23306, 23198]

## Задание №5
Выведите список ТОП-10 похожих товаров с названиями (Description) на экран

In [ ]:
df.loc[df['StockCode'].isin(top10_index_list), ['StockCode', 'Description']].drop_duplicates().set_index('StockCode')

,Description
StockCode,
22961,JAM MAKING SET PRINTED
22666,RECIPE BOX PANTRY YELLOW DESIGN
22722,SET OF 6 SPICE TINS PANTRY DESIGN
22720,SET OF 3 CAKE TINS PANTRY DESIGN
22993,SET OF 4 PANTRY JELLY MOULDS
23167,SMALL CERAMIC TOP STORAGE JAR
23165,LARGE CERAMIC TOP STORAGE JAR
23307,SET OF 60 PANTRY DESIGN CAKE CASES
23306,SET OF 36 DOILIES PANTRY DESIGN
